## This is a notebook that verify the communication of the pointing kernel with M2 (the rotator is also used)

In this notebook, we 
* point the telescope to a particular pointing, 
* make sure the mt telemetry is correct
* make sure that the zenith angle is picked up by M2 via the subscription
* check that M2 LUT forces vary with the zenith angle acccordingly

When we run this on the summit, we do not use the pointing component. We simply use a mount Controller to publish the mount elevation angle.

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from astropy.utils import iers
iers.conf.auto_download = False

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from lsst.ts.idl.enums import MTM2

plt.jet();

In [ ]:
#summit = 1 #use this for summit testing
summit = 0 #use this for NCSA

In [ ]:
if summit:
    os.environ["OSPL_URI"]="file:///home/bxin/WORK/ts_ddsconfig/config/ospl-sp.xml"
    os.environ["LSST_DDS_DOMAIN_ID"] = "11"
    print(os.environ["OSPL_URI"])
    print(os.environ["LSST_DDS_PARTITION_PREFIX"])
    print(os.environ["LSST_DDS_DOMAIN_ID"])
else:
    import os
    print(os.environ["OSPL_URI"])
    if os.environ.get("LSST_DDS_ALIGNER", "false") != "false":
        print("LSST_DDS_ALIGNER is mis-configured")

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
if not summit:
    await asyncio.sleep(10) #no need to wait on summit since we use the daemon
    ptg = salobj.Remote(script.domain, "MTPtg")
    rot = salobj.Remote(script.domain, "MTRotator")
mount = salobj.Remote(script.domain, "MTMount") #we still need it on the summit, to check (not generate) fake mount telemetry!
m2 = salobj.Remote(script.domain, "MTM2", exclude=['logMessage'])
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
if summit:
    await asyncio.gather(m2.start_task,
                         mount.start_task)
else:
    await asyncio.gather(ptg.start_task,
                         mount.start_task,
                         script.start_task, #                     
                         rot.start_task,
                         m2.start_task)

In [ ]:
if not summit:
    await ptg.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
sstate = await m2.evt_summaryState.aget(timeout=5)
print('staring with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))
dstate = await m2.evt_detailedState.aget(timeout=200)
print('m2 state', MTM2.DetailedState(dstate.detailedState), pd.to_datetime(dstate.private_sndStamp, unit='s'))
if not summit:
    state = await ptg.evt_summaryState.aget(timeout=5)
    print('ptg', salobj.State(state.summaryState),pd.to_datetime(state.private_sndStamp, unit='s'))
    state = await rot.evt_summaryState.aget(timeout=5)
    print('rot', salobj.State(state.summaryState),pd.to_datetime(state.private_sndStamp, unit='s'))
    state = await mount.evt_summaryState.aget(timeout=5)
    print('mount', state.summaryState,pd.to_datetime(state.private_sndStamp, unit='s'))

In [ ]:
#execute once to get the state UP one level
if sstate.summaryState == salobj.State.FAULT:
    await m2.cmd_clearErrors.set_start()
if sstate.summaryState == salobj.State.OFFLINE:
    await salobj.enterControl.set_start()    
if sstate.summaryState == salobj.State.STANDBY:
    await m2.cmd_start.set_start()
if sstate.summaryState == salobj.State.DISABLED:
    await m2.cmd_enable.set_start() 
sstate = await m2.evt_summaryState.aget(timeout=5)
print('starting with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

In [ ]:
#if ptg is in standby (5), enable it
if not summit:
    await salobj.set_summary_state(ptg, salobj.State.ENABLED)

Check the state of the system

In [ ]:
axialForces = await m2.tel_axialForce.aget(timeout=2)
tangentForces = await m2.tel_tangentForce.aget(timeout=2)

In [ ]:
m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
print("starting with Status of the M2 force balance system ---", m2ForceBalance.status, "----",
      pd.to_datetime(m2ForceBalance.private_sndStamp, unit='s'))
if not m2ForceBalance.status:
    await m2.cmd_switchForceBalanceSystem.set_start(status=True, timeout=10)
    m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
    print("Status of the M2 force balance system", m2ForceBalance.status)

### Do a slew (only for NCSA)

In [ ]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
alt = 40 * u.deg
az = 0 * u.deg

target_name="TMA motion test"
time_data = await ptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_ptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_ptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

print(curr_time_ptg.tai.value)

cmd_elaz = AltAz(alt=alt, az=az, 
                obstime=curr_time_ptg.tai, 
                location=mtcs.location)
cmd_radec = cmd_elaz.transform_to(ICRS)
await mtcs.slew_icrs(ra=cmd_radec.ra, dec=cmd_radec.dec, rot=0., rot_type=RotType.PhysicalSky)

In [ ]:
if not summit:
    mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
    rotStatus = await rot.evt_inPosition.aget(timeout=5.)
    print('Are we tracking?', mountStatus.elevation , mountStatus.azimuth , rotStatus.inPosition)

In [ ]:
if not summit:
    await ptg.cmd_stopTracking.set_start(timeout=5.)

### Check telemetry: list here what we want to check once the slew is done

In [ ]:
# Check that the mirror is in Position --  this check is a little more tricky if we are in closed loop. 
m2InPosition = await m2.evt_m2AssemblyInPosition.aget(timeout=10.)
print("Is the m2 in Position after a slew?", m2InPosition.inPosition)

In [ ]:
mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.angleActual)

### Alternatively, we could create a Controller object to send mount telemetry (for NCSA OR summit)

If we are to use the Controller object, we need to 

* take the mount out of the disabled/enabled states so that it stops sending telemetry data. (check the mount telemetry, via EFD or here in the notebook)
* Get fake telemetry ready (using mountTelGenerator.ipynb) - dAngle is figured out from m2_diagnostics_EFD.ipynb
* Then we need to point m2 to the fake mount elevation

In [ ]:
m2AngleSource = await m2.evt_inclinationTelemetrySource.aget(timeout=10.)
print("Inclinometer Source", MTM2.InclinationTelemetrySource(m2AngleSource.source))

In [ ]:
await m2.cmd_selectInclinationTelemetrySource(status = MTM2.inclinationTelemetrySource.MTMount)
#await m2.cmd_selectInclinationTelemetrySource(status = MTM2.inclinationTelemetrySource.OnBoard)

In [ ]:
m2AngleSource = await m2.evt_inclinationTelemetrySource.aget(timeout=10.)
print("Inclinometer Source", MTM2.InclinationTelemetrySource(m2AngleSource.source))

In [ ]:
m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
print("starting with Status of the M2 force balance system ---", m2ForceBalance.status, "----",
      pd.to_datetime(m2ForceBalance.private_sndStamp, unit='s'))
if not m2ForceBalance.status:
    await m2.cmd_switchForceBalanceSystem.set_start(status=True, timeout=10)
    m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
    print("Status of the M2 force balance system", m2ForceBalance.status)

In [ ]:
m2Angle = await m2.tel_zenithAngle.aget(timeout=10.)
print("elevation from the source", 90 - m2Angle.measured)

mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.angleActual)

try a different dAngle:

Now set it using mountTelGenerator.ipynb

In [ ]:
m2Angle = await m2.tel_zenithAngle.aget(timeout=10.)
print("elevation from the source", 90 - m2Angle.measured)

mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.angleActual)

#this gives enough time for us to collect later to be analyzed later
await asyncio.sleep(15)

## Stop the system: stop tracking, what else?

In [ ]:
await ptg.cmd_stopTracking.start(timeout=10.)